### Import Packages

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

### Import Datasets

In [2]:
iris = datasets.load_iris() # Classification
wine = datasets.load_wine() # Classification
breast_cancer = datasets.load_breast_cancer() # Classification
boston = datasets.load_boston() # Regression
diabetes = datasets.load_diabetes() # Regression

### Create Necessary Functions

In [3]:
def predict(X, w):
    return np.dot(X, w)

In [4]:
def loss(X, y, w, reg):
    f_x = predict(X, w)
    idx = np.where(np.abs(f_x - y) < 1)
    rtn = np.abs(f_x - y) - 1
    rtn[idx] = 0
    rtn + reg * np.linalg.norm(w)
    return np.sum(rtn)

In [23]:
def d_loss(X, y, w, reg):
    f_x = predict(X, w)
    idx = np.where(np.abs(f_x - y) >= 1)
    mult = np.zeros(X.shape) - 2 * reg * w
    mult[idx, :] = X[idx,:] * np.sign(w) * np.sign(X[idx,:])
    return np.sum(mult, axis = 0)

In [6]:
def distance(X, w):
    return np.abs(np.dot(X, w)) / np.linalg.norm(w)

### Create Plotting Function

In [19]:
def plot_2dim_db(X, y, w):
    
    plot_color = ['r' if y[i] > 0 else 'b' for i in range(len(y))]

    plt.xlim([np.min(X[:, 1]) - 1, np.max(X[:, 1]) + 1])
    plt.ylim([np.min(X[:, 2]) - 1, np.max(X[:, 2]) + 1])
    plt.scatter(X[:, 1], X[:, 2], marker='o')

    dim1_min = np.min([X[:, 1], X[:, 2]])
    dim1_max = np.max([X[:, 1], X[:, 2]])

    dim2_min = -1 * (dim1_min * w[1] + w[0]) / w[2]
    dim2_max = -1 * (dim1_max * w[1] + w[0]) / w[2]

    plt.plot([dim1_min, dim1_max], [dim2_min, dim2_max], color = 'black')
    
    margin = 1/np.linalg.norm(w)
    
    margin_plus_min = -1 * (dim1_min * w[1] + w[0] + margin) / w[2]
    margin_plus_max = -1 * (dim1_max * w[1] + w[0] + margin) / w[2]
    
    plt.plot([dim1_min, dim1_max], [margin_plus_min, margin_plus_max], color = 'grey', linestyle = 'dashed')
    
    margin_minus_min = -1 * (dim1_min * w[1] + w[0] - margin) / w[2]
    margin_minus_max = -1 * (dim1_max * w[1] + w[0] - margin) / w[2]
    
    plt.plot([dim1_min, dim1_max], [margin_minus_min, margin_minus_max], color = 'grey', linestyle = 'dashed')

### Inititalize Variables

In [24]:
train_X, test_X, train_y, test_y = train_test_split(diabetes['data'][:,4:6], diabetes['target'],
                                                    test_size = .3, random_state = 0)

scale_X = StandardScaler().fit(train_X)
train_X = scale_X.transform(train_X)
test_X = scale_X.transform(test_X)

scale_y = StandardScaler().fit(train_y.reshape(-1, 1))
train_y = scale_y.transform(train_y.reshape(-1, 1))[:,0]
test_y = scale_y.transform(test_y.reshape(-1, 1))[:,0]

train_X = np.insert(train_X, 0, 1, axis = 1)
test_X = np.insert(test_X, 0, 1, axis = 1)

w = np.ones(shape = train_X.shape[1]) * -1
# w = np.array([1, -1, 1])
lr = .000001

reg = 0

### Train Model

In [25]:
for _ in range(100000):
    grad_loss = d_loss(train_X, train_y, w, reg)
    w = w - lr * grad_loss[0:len(w)]

# grad_loss = d_loss(train_X, train_y, w, reg)

# print(w)
# print(loss(train_X, train_y, w, reg))
# print(grad_loss)

# w = w - lr * grad_loss[0:len(w)]
# plot_2dim_db(train_X, train_y, w)


### Calculate Accuracy

In [26]:
# plot_2dim_db(test_X, test_y, w)
# print(predict(train_X, w))
# print(train_y)

np.sqrt(np.mean((predict(test_X, w) - test_y) ** 2)) * scale_y.scale_

array([71.42468937])